In [ ]:
import subprocess
import pandas as pd
import numpy as np
import gzip
import os

In [ ]:
# Load the full GWAS catalog
df = pd.read_csv("./data/vcf/gwas-catalog-associations.tsv", sep='\t', low_memory=False) # 2018-12-27

# Filter for disease
disease = 'type 2 diabetes'
disease_file = disease.replace(" ", "_")
df_disease = df[df['DISEASE/TRAIT'].str.contains(disease, case=False, na=False)]

# Drop rows missing position info
df_disease = df_disease.dropna(subset=['CHR_ID', 'CHR_POS'])

# Convert to VCF-style ids and positions
df_disease['CHR_ID'] = df_disease['CHR_ID'].astype(str).str.split(';').str[0] # Keep only the first value before any semicolon
df_disease['CHR_POS'] = df_disease['CHR_POS'].astype(str).str.split(';').str[0] # Keep only the first value before any semicolon
df_disease = df_disease[df_disease['CHR_POS'].astype(str).str.match(r'^\d+$')]
df_disease['CHR_POS'] = df_disease['CHR_POS'].astype(int)
df_disease = df_disease[df_disease['DISEASE/TRAIT']=='Type 2 diabetes'] # Keep only the rows that are just type 2 diabetes

# Create a directory only if it doesn't exist
folder_path = f"./data/vcf/{disease_file}"
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    
# Save to tsv
df_disease[['CHR_ID', 'CHR_POS']].to_csv(f"./data/vcf/{disease_file}/{disease_file}_variants.tsv", sep='\t', index=False, header=False)

In [ ]:
# Filter each chromosome VCF file for variants
for i in range(1, 23):
    vcf_filter_command = [
        "bcftools", "view",
        "-R", f"./data/vcf/{disease_file}/{disease_file}_variants.tsv",
        f"./data/vcf/ALL.chr{i}.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz",
        "-Oz", "-o", f"./data/vcf/{disease_file}/{disease_file}_chr{i}.vcf.gz"
    ]

    subprocess.run(vcf_filter_command, check=True)

# Files to combine
vcf_file = f'ls ./data/vcf/{disease_file}/{disease_file}_chr*.vcf.gz > ./data/vcf/{disease_file}/vcf_list.txt'
subprocess.run(vcf_file, shell=True, check=True)

# Combine all filtered files into one VCF file
vcf_concat_command = [
    "bcftools", "concat",
    "-f", f"./data/vcf/{disease_file}/vcf_list.txt",
    "-Oz", "-o", f"./data/vcf/{disease_file}/{disease_file}_all.vcf.gz"
]
subprocess.run(vcf_concat_command, check=True)

# Filter VCF for substitutions only
sub_filter = [
    "bcftools", "view",
    "-e", 'ALT ~ "<"',
    f"./data/vcf/{disease_file}/{disease_file}_all.vcf.gz",
    "-Oz",
    "-o", f"./data/vcf/{disease_file}/{disease_file}_all_substitution.vcf.gz"
]
subprocess.run(sub_filter, check=True)

In [ ]:
# Read vcf as DataFrame for sanity check

# Filepath
vcf_path = f"./data/vcf/{disease_file}/{disease_file}_all_substitution.vcf.gz"

# Find the header line
with gzip.open(vcf_path, 'rt') as f:
    for i, line in enumerate(f):
        if line.startswith("#CHROM"):
            header_line_num = i
            break

# Read the file as DataFrame
df_disease_vcf = pd.read_csv(
    vcf_path,
    sep='\t',
    compression='gzip',
    skiprows=header_line_num,
    header=0
)

df_disease_vcf